In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc = {'figure.figsize':(16,9)}, font_scale = 1.5)

In [5]:
df_375 = pd.read_excel(io = "time_series_375_preprocess_en.xlsx")
df_110 = pd.read_excel(io = "time_series_test_110_preprocess_en.xlsx")

df_110

,PATIENT_ID,RE_DATE,Lactate dehydrogenase,Hypersensitive c-reactive protein,(%)lymphocyte,Admission time,Discharge time,outcome
0,1.0,2020-02-10 11:15:00,NaN,NaN,28.0,2020-02-09 10:47:24,2020-02-21 14:51:12,0
1,NaN,2020-02-10 12:07:00,183.0,5.1,NaN,2020-02-09 10:47:24,2020-02-21 14:51:12,0
2,NaN,2020-02-13 09:35:00,NaN,NaN,28.6,2020-02-09 10:47:24,2020-02-21 14:51:12,0
3,NaN,2020-02-13 10:06:00,194.0,2.7,NaN,2020-02-09 10:47:24,2020-02-21 14:51:12,0
4,2.0,2020-02-09 06:56:00,NaN,NaN,28.3,2020-02-09 03:39:47,2020-02-21 11:57:40,0
...,...,...,...,...,...,...,...,...
752,NaN,2020-02-05 12:36:00,562.0,78.1,NaN,2020-02-05 00:27:57,2020-02-22 14:11:47,0
753,NaN,2020-02-11 09:40:00,NaN,NaN,9.7,2020-02-05 00:27:57,2020-02-22 14:11:47,0
754,NaN,2020-02-11 10:04:00,438.0,NaN,NaN,2020-02-05 00:27:57,2020-02-22 14:11:47,0
755,NaN,2020-02-17 10:00:00,430.0,31.2,NaN,2020-02-05 00:27:57,2020-02-22 14:11:47,0
